# Repositories Smoke Testing (Stage 3)

## Objective
Validate repository contracts (`CandidateRepository`, `ResumeRepository`, `ResumeSectionRepository`) in isolation.

## Prerequisites
1. Local DB must be reachable via `DATABASE_URL`.
2. Migrations should already be applied.
3. This notebook writes test records prefixed with `nb_test_`.


In [ ]:
from datetime import datetime, timezone

from notebooks._utils import print_checkpoint
from src.storage.db import get_session
from src.storage.repositories import CandidateRepository, ResumeRepository, ResumeSectionRepository

RUN_DB_SMOKE = True  # set False to skip DB writes in this notebook

session = get_session()
print("Session created")
print_checkpoint("setup complete")


## 1) Candidate create + get_or_create idempotency


In [ ]:
if not RUN_DB_SMOKE:
    print("Skipping DB smoke tests (RUN_DB_SMOKE=False)")
else:
    c_repo = CandidateRepository(session)
    ext_id = f"nb_test_candidate_{datetime.now(timezone.utc).strftime('%Y%m%d%H%M%S')}"

    c1, created1 = c_repo.get_or_create_by_external_id(external_id=ext_id, name="NB Test Candidate")
    c2, created2 = c_repo.get_or_create_by_external_id(external_id=ext_id, name="NB Test Candidate")

    assert created1 is True, "first get_or_create should create"
    assert created2 is False, "second get_or_create should reuse"
    assert c1.id == c2.id, "idempotent lookup returned different records"

    print("candidate_id:", c1.id)
    print_checkpoint("candidate idempotency assertions passed")


## 2) Resume create + roundtrip integrity


In [ ]:
if RUN_DB_SMOKE:
    r_repo = ResumeRepository(session)
    source_file = f"nb_test_resume_{datetime.now(timezone.utc).strftime('%Y%m%d%H%M%S')}.pdf"

    resume = r_repo.create(
        candidate_id=c1.id,
        source_file=source_file,
        raw_text="nb_test raw text",
        parsed_json={"clean_text": "nb_test clean", "section_names": ["summary"]},
        language="en",
    )

    fetched = r_repo.get_by_source_file(source_file)
    assert fetched is not None, "resume roundtrip failed"
    assert fetched.source_file == source_file
    assert fetched.language == "en"
    assert isinstance(fetched.parsed_json, dict)

    print("resume_id:", resume.id)
    print_checkpoint("resume roundtrip assertions passed")


## 3) Resume section persistence


In [ ]:
if RUN_DB_SMOKE:
    s_repo = ResumeSectionRepository(session)
    section = s_repo.create(
        resume_id=resume.id,
        section_type="summary",
        content="nb_test section content",
        metadata_json={"origin": "notebook_smoke"},
        tokens=4,
    )

    assert section.id is not None
    assert section.section_type == "summary"
    assert section.tokens == 4

    print("section_id:", section.id)
    print_checkpoint("section persistence assertions passed")


## 4) Commit and teardown guidance

1. Commit only notebook test records you intentionally inserted.
2. Test records are prefixed with `nb_test_` for easy cleanup.
3. You can roll back instead of commit while iterating.


In [ ]:
if RUN_DB_SMOKE:
    session.commit()
    print("Committed notebook smoke records")
else:
    session.rollback()

session.close()
print_checkpoint("session closed")


## Summary (fill after run)

- Candidate idempotency: pass/fail
- Resume roundtrip: pass/fail
- Section persistence: pass/fail
- DB connectivity issues observed: yes/no


## Next actions

1. If any assertion fails, reproduce with a focused unit test in `tests/`.
2. Keep notebook-only records identifiable via `nb_test_` prefix.
3. If schema issues appear, verify migrations and model alignment first.
